In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from datetime import timedelta
from functools import reduce
from tqdm import tqdm
import lightgbm as lgbm
import mlb
import matplotlib.pyplot as plt

import plotly.express as px

features that we have now

In [ ]:
# feature_cols = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
#        'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
#        'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
#        'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
#        'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
#        'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
#        'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
#        'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
#        'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
#        'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
#        'groundOutsPitching', 'runsPitching', 'doublesPitching',
#        'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
#        'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
#        'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
#        'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
#        'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
#        'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
#        'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
#        'inheritedRunnersScored', 'catchersInterferencePitching',
#        'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
#        'assists', 'putOuts', 'errors', 'chances','target1_mean',
#  'target1_median',
#  'target1_std',
#  'target1_min',
#  'target1_max',
#  'target1_prob',
#  'target2_mean',
#  'target2_median',
#  'target2_std',
#  'target2_min',
#  'target2_max',
#  'target2_prob',
#  'target3_mean',
#  'target3_median',
#  'target3_std',
#  'target3_min',
#  'target3_max',
#  'target3_prob',
#  'target4_mean',
#  'target4_median',
#  'target4_std',
#  'target4_min',
#  'target4_max',
#  'target4_prob']
# feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
#        'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
#        'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
#        'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
#        'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
#        'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
#        'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
#        'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
#        'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
#        'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
#        'groundOutsPitching', 'runsPitching', 'doublesPitching',
#        'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
#        'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
#        'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
#        'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
#        'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
#        'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
#        'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
#        'inheritedRunnersScored', 'catchersInterferencePitching',
#        'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
#        'assists', 'putOuts', 'errors', 'chances','target1_mean',
#  'target1_median',
#  'target1_std',
#  'target1_min',
#  'target1_max',
#  'target1_prob',
#  'target2_mean',
#  'target2_median',
#  'target2_std',
#  'target2_min',
#  'target2_max',
#  'target2_prob',
#  'target3_mean',
#  'target3_median',
#  'target3_std',
#  'target3_min',
#  'target3_max',
#  'target3_prob',
#  'target4_mean',
#  'target4_median',
#  'target4_std',
#  'target4_min',
#  'target4_max',
#  'target4_prob',
#     'target1']

In [ ]:
BASE_DIR = Path('../input/mlb-player-digital-engagement-forecasting')
TRAIN_DIR = Path('../input/mlb-pdef-train-dataset')

#### In the current data, we use only 
> rosters_train.pkl <br>
> nextDayPlayerEngagement_train.pkl <br>
> playerBoxScores_train.pkl <br>
> player_target_stats.csv

In [ ]:
players = pd.read_csv(BASE_DIR / 'players.csv')

rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')
followers = pd.read_pickle(TRAIN_DIR / 'playerTwitterFollowers_train.pkl')
team_followers = pd.read_pickle(TRAIN_DIR / 'teamTwitterFollowers_train.pkl')
team_followers = team_followers.rename(columns={'numberOfFollowers': 'teamFollowers'})
scores = pd.read_pickle(TRAIN_DIR / 'playerBoxScores_train.pkl')
scores = scores.groupby(['playerId', 'date']).sum().reset_index()

In [ ]:
targets

In [ ]:
best_sub = pd.read_csv('../input/endtoendsolution/submission.csv')
best_sub

In [ ]:
import seaborn as sns

In [ ]:
targets_cols = ['target1', 'target2', 'target3', 'target4']

for target in targets_cols:
    sns.distplot(targets[target])
    plt.title(target)
    plt.show()

In [ ]:
targets_cols = ['target1', 'target2', 'target3', 'target4']

for target in targets_cols:
    sns.distplot(best_sub[target])
    plt.title(target)
    plt.show()

# Rosters EDA

- playerId - Unique identifier for a player.
- gameDate - dat of the game
- teamId - teamId that player is on that date.
- statusCode - Roster status abbreviation.
- status - Descriptive roster status.

In [ ]:
rosters.head()

In [ ]:
rosters.status.unique()

In [ ]:
statusCode = rosters.statusCode.value_counts()

fig = px.bar(x=statusCode.index, y=np.log(statusCode.values), text=statusCode.values,
            labels={'x': 'roster status code', 'y': 'values'},
            title='Number of players under each roster status(log)', 
            color=statusCode.index)

fig.show()

In [ ]:
rosters['year'] = pd.to_datetime(rosters['date'], format = '%Y%m%d').dt.year

In [ ]:
status_change_per_year = pd.DataFrame(rosters.groupby('year')['statusCode'].value_counts())
status_change_per_year.columns = ['count']
status_change_per_year = status_change_per_year.reset_index()

fig = px.bar(x=status_change_per_year['statusCode'], y=np.log(status_change_per_year['count']), 
             text=status_change_per_year['count'],
             animation_frame=status_change_per_year['year'], animation_group=status_change_per_year['count'],
             labels={'x': 'roster status code', 'y': 'values'},
             title='Number of players under each roster status per year(log)', 
             color=status_change_per_year['statusCode'])

fig.show()

# Player twitter followers EDA

Twitter following data was collected by MLB from Twitter APIs for Major League players, on the first of each month, dating back to 1/1/2018. This dataset is not exhaustive of all players over all months, as not every player has/had a Twitter account, players may create/delete/reinstate accounts at random, or other scenarios preventing follower data from being collected on a given date. <br>

- date - Date of follower count.
- playerId - Unique identifier for a player.
- playerName
- accountName - Name on player twitter account.
- twitterHandle - Player twitter handle.
- numberOfFollowers - Number of followers

In [ ]:
followers.head()

In [ ]:
followers.shape

In [ ]:
followers.playerName.unique().shape

In [ ]:
most_famous_player = followers.groupby('playerName')['numberOfFollowers'].max().sort_values(ascending=False)[:20]

fig = px.bar(x=most_famous_player.index, y=most_famous_player.values, text=most_famous_player.values,
            labels={'x': 'Name', 'y': 'followers'},
            title='Top 20 the most famous players by tweeter followers', 
            color=most_famous_player.index)
fig.show()

In [ ]:
followers['date'] = pd.to_datetime(followers['date'], format = '%Y%m%d')

In [ ]:
followers_growth = followers.groupby(['playerId', 'date']).agg({'numberOfFollowers':'min'}).reset_index()

In [ ]:
min_followers = \
    pd.DataFrame(followers_growth.groupby('playerId').agg({'numberOfFollowers':'min'})['numberOfFollowers']).reset_index()
min_followers.columns = ['playerId', 'min_f']

max_followers = \
    pd.DataFrame(followers_growth.groupby('playerId').agg({'numberOfFollowers':'max'})['numberOfFollowers']).reset_index()
max_followers.columns = ['playerId', 'max_f']

In [ ]:
def get_growth(min_max_dataset):
    A = min_max_dataset['min_f']
    B = min_max_dataset['max_f']
    return (B-A)/A*100

In [ ]:
min_max_followers = min_followers.merge(max_followers, on='playerId', how='left')
min_max_followers['growth%'] = get_growth(min_max_followers)
min_max_followers['growth_num'] = min_max_followers['max_f']-min_max_followers['min_f']
min_max_followers = min_max_followers.sort_values(by=['growth_num', 'growth%'], ascending=False)[:20]

In [ ]:
min_max_followers

In [ ]:
fig = px.bar(x=min_max_followers['playerId'], y=min_max_followers['growth_num'], 
             text=round(min_max_followers['growth%']),
             labels={'x': 'playerId', 'y': 'followers'},
             title='Top 20 fastest growing players by tweeter followers', 
             color=min_max_followers['playerId'])
fig.show()

# Team followers EDA

Twitter following data was collected by MLB from Twitter APIs for all 30 Major League teams, on the first of each month, dating back to 1/1/2018. <br>

- date - Date of follower count.
- teamId - Unique identifier for a team.
- teamName
- accountName - Name on team twitter account.
- twitterHandle - Team twitter handle.
- numberOfFollowers - Number of followers.

In [ ]:
team_followers.head()

In [ ]:
team_followers.shape

In [ ]:
most_famous_teams = team_followers.groupby('teamName')['teamFollowers'].max().sort_values(ascending=False)

fig = px.bar(x=most_famous_teams.index, y=most_famous_teams.values, text=most_famous_teams.values,
            labels={'x': 'Team name', 'y': 'followers'},
            title='The most famous teams by tweeter followers', 
            color=most_famous_teams.index)
fig.show()

# Player Box Scores EDA

- home - Binary, 1 if home team, 0 if away.
- gamePk - unique identifier for game.
- gameDate
- gameTimeUTC - First pitch time in UTC.
- teamId - Unique identifier for team.
- teamName
- playerId - Unique identifier for player.
- playerName
- jerseyNum
- positionCode - Number position code, details are here.
- positionName - Text position display, details are here.
- positionType - Position group, details are here.
- battingOrder - Format: “###”, where the first digit indicates the batting order spot, and the second two digits indicate the sequence in which that player occupied that batting order spot. Examples: “300”, which indicates the starter in the third spot in the batting order; “903,” which indicates the fourth player (after 900, 901 and 902) occupy the ninth spot in the batting order. Only populates if appeared in game.
- gamesPlayedBatting - 1 if player entered the game as a batter, runner or fielder.
- flyOuts - Game total fly outs.
- groundOuts - Game total ground outs.
- runsScored - Game total runs scored.
- doubles - Game total doubles.
- triples - Game total triples.
- homeRuns - Game total home runs.
- strikeOuts - Game total strike outs.
- baseOnBalls - Game total walks.
- intentionalWalks - Game total intentional walks.
- hits - Game total hits.
- hitByPitch - Game total hit by pitches.
- atBats - Game total at-bats.
- caughtStealing - Game total caught stealing.
- stolenBases - Game total stolen bases.
- groundIntoDoublePlay - Game total double plays grounded into.
- groundIntoTriplePlay - Game total triple plays grounded into.
- plateAppearances - Game total plate appearances.
- totalBases - Game total bases.
- rbi - Game total runs batted in.
- leftOnBase - Game total runners left on base.
- sacBunts - Game total sacrifice bunts.
- sacFlies - Game total sacrifice flies.
- catchersInterference - Game total catchers interference reached on.
- pickoffs - Game total number of times picked off base.
- gamesPlayedPitching- Binary, 1 if player entered the game as a pitcher.
- gamesStartedPitching- Binary, 1 if player was game’s starting pitcher.
- completeGamesPitching - Binary, 1 if credited with complete game.
- shutoutsPitching - Binary, 1 if credited with shutout.
- winsPitching - Binary, 1 if credited with win.
- lossesPitching - Binary, 1 if credited with loss.
- flyOutsPitching - Game total of flyouts allowed.
- airOutsPitching - Game total of air outs (flyouts + popouts) allowed.
- groundOutsPitching - Game total ground outs allowed.
- runsPitching - Game total runs allowed.
- doublesPitching - Game total doubles allowed.
- triplesPitching - Game total triples allowed.
- homeRunsPitching - Game total home runs allowed.
- strikeOutsPitching - Game total strike outs allowed.
- baseOnBallsPitching - Game total walks allowed.
- intentionalWalksPitching - Game total intentional walks allowed.
- hitsPitching - Game total hits allowed.
- hitByPitchPitching - Game total hit by pitches allowed.
- atBatsPitching - Game total at-bats against.
- caughtStealingPitching - Game total caught stealing against.
- stolenBasesPitching - Game total stolen bases allowed.
- inningsPitched - Game total innings pitched.
- saveOpportunities - Binary, 1 if credited with save opportunity.
- earnedRuns - Game total earned runs allowed.
- battersFaced - Game total batters faced.
- outsPitching - Game total outs recorded.
- pitchesThrown - Game total number of pitches thrown.
- balls - Game total balls thrown.
- strikes - Game total strikes thrown.
- hitBatsmen - Game total hit batters.
- balks - Game total balks.
- wildPitches - Game total number of wild pitches thrown.
- pickoffsPitching - Game total number of pickoffs.
- rbiPitching - Game total number of runs batted in allowed.
- inheritedRunners - Game total of inherited runners assumed.
- inheritedRunnersScored- Game total of inherited runners that scored.
- catchersInterferencePitching - Game total of catcher’s interference occurred by battery.
- sacBuntsPitching - Game total sacrifice bunts allowed.
- sacFliesPitching - Game total sacrifice flies allowed.
- saves - Binary, 1 if credited with save.
- holds - Binary, 1 if credited with hold.
- blownSaves - Binary, 1 if credited with blown save.
- assists - Game total number of assists.
- putOuts - Game total number of putouts.
- errors - Game total number of errors.
- chances - Game total fielding chances.

In [ ]:
scores.head()

In [ ]:
st = scores.columns[2:45]
nd = scores.columns[45:]

In [ ]:
scores[st].describe()

In [ ]:
scores[nd].describe()

# Awards EDA

In [ ]:
awards = pd.read_csv(TRAIN_DIR / 'awards_train.csv')

In [ ]:
awards['awardDateMonth'] = pd.to_datetime(awards['awardDate'], format = '%Y-%m-%d').dt.month
awards['awardDateYear'] = pd.to_datetime(awards['awardDate'], format = '%Y-%m-%d').dt.year

In [ ]:
awards.head()

In [ ]:
most_awarded_player_per_year = pd.DataFrame(awards.groupby('awardDateYear')['playerName'].value_counts())
most_awarded_player_per_year.columns = ['num_of_awards']
most_awarded_player_per_year = most_awarded_player_per_year.reset_index()
year2018 = most_awarded_player_per_year[most_awarded_player_per_year['awardDateYear'] == 2018][:20]
year2019 = most_awarded_player_per_year[most_awarded_player_per_year['awardDateYear'] == 2019][:20]
year2020 = most_awarded_player_per_year[most_awarded_player_per_year['awardDateYear'] == 2020][:20]
year2021 = most_awarded_player_per_year[most_awarded_player_per_year['awardDateYear'] == 2021][:20]
most_awarded_player_per_year = year2018.append([year2019, year2020, year2021])

fig = px.bar(x=most_awarded_player_per_year['playerName'], y=most_awarded_player_per_year['num_of_awards'], 
             text=most_awarded_player_per_year['num_of_awards'],
             animation_frame=most_awarded_player_per_year['awardDateYear'],
             animation_group=most_awarded_player_per_year['num_of_awards'],
             labels={'y': 'number of awards', 'x': ' '},
             title='Top the most awarded players per year', 
             color=most_awarded_player_per_year['playerName'])

fig.show()

In [ ]:
most_frequent_award = awards.awardName.value_counts()[:25]

fig = px.pie(most_frequent_award, values=most_frequent_award.values, names=most_frequent_award.index,
             title='Most frequent awards')
fig.show()

In [ ]:
def plot_awards(playerid=477132):
    if len(awards[awards.playerId==playerid]['awardDateYear'].value_counts()) > 0:
        awards[awards.playerId==playerid]['awardDateYear'].value_counts().plot(kind = 'bar')
        plt.show()

In [ ]:
plot_awards(477132)

# Conclusion

- about rosters: most players play under the A status and I don't think that this can help in training the model.
- about players followers: it is logical that with high popularity, the interest of fans will be greater than with a low number of readers. I also showed the growth of followers on the graph, this can serve as a feature, as it seems to me, but the correctness still needs to be improved.
- about teams followers: the players in the most popular teams were followed on average more than in the unpopular ones, and this can also become a feature.
- about awards: here, too, it is interesting to do. Players receive awards and fan engagement increases, if this award is some kind of prestigious, then interest rises even more.

I also want to make an EDA based on other data such as standings, teamBoxScores, transactions, games.

# Events EDA

* gamePk - unique identifier for game.
* gameDate
* gameTimeUTC - First pitch time in UTC.
* season
* gameType - Game type, various types can be found here.
* playId - Statcast play guid.
* eventId
* inning - Inning AB occurred.
* halfInning- “top” or “bottom” inning indicator.
* homeScore - Home score at start of the event.
* awayScore - Away score at start of the event.
* menOnBase - Split used if runner on base – ie (RISP, Empty).
* atBatIndex - At-Bat index number, begins with 0.
* atBatDesc - Play-by-play description of AB.
* atBatEvent - Event type result of the atBat, various types can be found here.
* hasOut - Binary, 1 if runner is out on play.
* pitcherTeamId - Unique identifier for pitching team.
* isPitcherHome - Binary, 1 if pitcher is home team.
* pitcherTeam - team name of pitching team.
* hitterTeamId - Unique identifier for hitting team.
* hitterTeam - Team name of hitting team.
* pitcherId
* pitcherName
* isStarter - Binary, 1 if player was game’s starting pitcher.
* pitcherHand - Player throw hand: “L”,”R”.
* hitterId
* hitterName
* batSide - Player bat side: “L”,”R”.
* pitchNumber - Pitch sequence number of AB.
* balls - Ball count after the event.
* strikes - Strike count after the event.
* isGB - Binary, 1 if at bat is a ground ball.
* isLD - Binary, 1 if at bat is a line drive.
* isFB - Binary, 1 if at bat is a fly ball.
* isPU - Binary, 1 if at bat is a pop up.
* launchSpeed - Measured speed of the hit ball.
* launchAngle - Vertical angle relative to the horizon at which the hit was launched.
* totalDistance - Total distance ball traveled.
* event - Events that may occur during an at-bat: strikeout, groundout, substitutions, stolen base, wild pitch, game advisory, etc.
* description - Text description of event.
* rbi - Number of runs batted in during AB.
* pitchType - Pitch type classification code, various types can be found here.
* call - Pitch or throw result classification code, various types can be found here.
* outs - Current/Final outs of AB.
* inPlay - True/False if ball was hit into play.
* isPaOver - Binary, 1 if this event is the end of the plate appearance.
* startSpeed - Speed in MPH of the ball at 50 feet in front of home plate.
* endSpeed - Speed in MPH of the ball as it crosses the front edge of home plate (0,0 in the x axis).
* nastyFactor - Evaluates several properties of each pitch, and rates the "nastiness" of the pitch on a scale from 0-100.
* breakAngle - Degrees clockwise (batter’s view) that the plane of the pitch deviates from the vertical.
* breakLength - Max distance that the pitch separates from the straight line between pitch start and pitch end.
* breakY - Distance from home plate where the break is greatest.
* spinRate - Rate of spin on the ball after it was released by pitcher in RPMs.
* spinDirection - The axis of rotation for the ball at release given as an angle that reflects how the spin will influence the ball trajectory. Pure back-spin is 180 degrees, pure side-spin that puts the ball to the 1b side is 90 degrees, pure-side spin that pulls the ball to the 3b side is 270 degrees, and pure top-spin is 0 or 360 degrees.
* pX - Horizontal position in feet of the ball as it crosses the front axis of home plate.
* pZ - Vertical position in feet above home plate of the ball as it crosses the front axis of home plate.
* aX - Ball acceleration on the z axis.
* aY - Ball acceleration on the y axis.
* aZ - Ball acceleration on the z axis.
* pfxX - Horizontal movement of the ball in inches.
* pfxZ - Vertical movement of the ball in inches.
* vX0 - Velocity of the ball from the x axis.
* vY0 - Velocity of the ball from the y axis; this is negative because 0,0,0 is behind the batter and the ball travels from pitcher mound towards 0,0,0.
* vZ0 - Velocity of the ball from the z axis.
* x - X coordinate where pitch crossed front of home plate.
* y - Y coordinate where pitch crossed front of home plate.
* x0 - Coordinate location of the ball at the point it was released from the pitchers hand on the x axis (time = 0).
* y0 - Coordinate location of the ball at the point it was released from the pitchers hand on the y axis (time = 0).
* z0 - Coordinate location of the ball at the point it was released from the pitchers hand on the z axis (time = 0).
* type - Type of event, either ‘pitch’ or ‘action’
* zone - Zone location number - see image below.

In [ ]:
events = pd.read_csv('../input/mlb-pdef-train-dataset/events_train.csv')
events.head()

In [ ]:
events.event.unique()

In [ ]:
events['year'] = pd.to_datetime(events.date, format = '%Y%m%d').dt.year
events['month'] = pd.to_datetime(events.date, format = '%Y%m%d').dt.month

In [ ]:
events2020 = events[events['year'] == 2020]
events2020.month.unique()

In [ ]:
most_frequent_events = events.event.value_counts()[:25]

fig = px.pie(most_frequent_events, values=most_frequent_events.values, names=most_frequent_events.index,
             title='Most frequent events')
fig.show()

In [ ]:
events.shape

In [ ]:
events.gameType.unique()

* R : Regular Season
* F : Wild Card Game
* D : Division Series
* L : League Championship Series
* W : World Series

In [ ]:
gameType = events.gameType.value_counts()
text = {'R' : 'Regular Season',
               'F' : 'Wild Card Game',
               'D' : 'Division Series',
               'L' : 'League Championship Series',
               'W' : 'World Series'}

fig = px.bar(gameType, x=gameType.index, y=np.log(gameType.values),
             color=gameType.index, text=text,
             title='log number of game types',
             labels={'index':'type', 'y':'value'})

fig.show()

In [ ]:
inning = events.inning

fig = px.histogram(inning, nbins=20,
                   title='distribution of innings')

fig.show()

#

# Target EDA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
sns.set(font_scale = 1.5)
fig, axs = plt.subplots(2,2, figsize = (20, 10))
sns.kdeplot(ax=axs[0,0], data=targets['target1'])
sns.kdeplot(ax=axs[0,1], data=targets['target2'])
sns.kdeplot(ax=axs[1,0], data=targets['target3'])
sns.kdeplot(ax=axs[1,1], data=targets['target4'])
bbox = axs[0,0].get_position()
bbox2 = axs[0,1].get_position()

center=(bbox2.x1) * 0.4 + (bbox.x1) * 0.25
plt.suptitle('Distribution of targets', x = center)


In [ ]:
def draw_kde_plot(col = 'target1'):
    sns.set_style('whitegrid')
    sns.set(font_scale = 1.5)
    fig, axs = plt.subplots(2,2, figsize = (15, 10))
    g = sns.kdeplot(ax=axs[0,0], data=targets[col])
    g.set_xlabel('original')
    g = sns.kdeplot(ax=axs[0,1], data=targets[col]**2)
    g.set_xlabel('squared')
    g = sns.kdeplot(ax=axs[1,0], data=targets[col]**4)
    g.set_xlabel('power 4')
    g = sns.kdeplot(ax=axs[1,1], data = np.log(targets[col]+1))
    g.set_xlabel('log')



    bbox = axs[0,0].get_position()
    bbox2 = axs[0,1].get_position()
    center=(bbox2.x1) * 0.4 + (bbox.x1) * 0.25
    plt.suptitle(f'Transformation of {col}', x = center)
    plt.tight_layout()


# **Target2 has highest Skewness**


In [ ]:
for col in ['target1', 'target2', 'target3', 'target4']:
    draw_kde_plot(col)

In [ ]:
sns.set_style('whitegrid')
sns.set(font_scale = 1.5)


fig, axs = plt.subplots(1,1, figsize = (20,8))
sns.lineplot(ax=axs, x = np.arange(1,10001),
             y = targets.sample(10000, random_state=500)['target1'],
             legend='full', label = 'target1')
sns.lineplot(ax=axs, x = np.arange(1,10001),
             y = targets.sample(10000, random_state=500)['target2'],
             legend='full', label = 'target2')
sns.lineplot(ax=axs, x = np.arange(1,10001), 
             y = targets.sample(10000, random_state=500)['target3'], 
             legend='full', label = 'target3')
sns.lineplot(ax=axs,x = np.arange(1,10001), 
             y = targets.sample(10000, random_state=500)['target4'], 
             legend='full', label = 'target4')

bbox = axs.get_position()
center=0.5*(bbox.x1)
plt.suptitle('Comparision of targets', x = center)



In [ ]:
sns.set_style('ticks')
sns.set(font_scale = 1.5)


fig, axs = plt.subplots(2,2, figsize = (20,8))
sns.lineplot(ax=axs[0,0], x = np.arange(1,10001),
             y = targets.sample(10000, random_state=500)['target1'],
             legend='full', label = 'target1')
sns.lineplot(ax=axs[0,1], x = np.arange(1,10001),
             y = targets.sample(10000, random_state=500)['target2'],
             legend='full', label = 'target2')
sns.lineplot(ax=axs[1,0], x = np.arange(1,10001), 
             y = targets.sample(10000, random_state=500)['target3'], 
             legend='full', label = 'target3')
sns.lineplot(ax=axs[1,1], x = np.arange(1,10001), 
             y = targets.sample(10000, random_state=500)['target4'], 
             legend='full', label = 'target4')

plt.title('Comparision of targets, side by side view')


In [ ]:
targets['year'] = pd.to_datetime(targets['date'], format = '%Y%m%d').dt.year

# We have less data for year 4, since we need to predict for the future

May be we should have different validation strategy

In [ ]:
targets['year'].value_counts().plot(kind = 'bar')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style="whitegrid")
sns.set(font_scale = 2)
sns.color_palette("Set2")



fig, axs = plt.subplots(4,1, figsize = (20,20))
sns.lineplot(ax=axs[0], 
             x = np.arange(1,10001),
             data = targets.sample(10000, random_state=100),
             y = 'target1',
             hue = 'year',
             palette='tab10',
             linewidth=2.5)

sns.lineplot(ax=axs[1], 
             x = np.arange(1,10001),
             data = targets.sample(10000, random_state=100),
             y = 'target2',
             hue = 'year',
             palette='tab10',
             linewidth=2.5)

sns.lineplot(ax=axs[2], 
             x = np.arange(1,10001),
             data = targets.sample(10000, random_state=100),
             y = 'target3',
             hue = 'year',
             palette='tab10',
             linewidth=2.5)

sns.lineplot(ax=axs[3], 
             x = np.arange(1,10001),
             data = targets.sample(10000, random_state=100),
             y = 'target4',
             hue = 'year',
             palette='tab10',
             linewidth=2.5)

bbox = axs[0].get_position()
center=0.5*(bbox.x1)
plt.suptitle('targets over years', x = center)

In [ ]:

def plot_target_for_player(col = 'target1', playerid = 683734):
    
    sns.set(style="whitegrid")
    sns.set(font_scale = 2)
    sns.color_palette("Set2")

    fig, axs = plt.subplots(1,1, figsize = (20,8))

    sns.lineplot(ax=axs, x = np.arange(365),
                 data = targets[((targets.year==2018) & (targets.playerId==playerid))],
                 y = col,
                 label = '2018',
                 linewidth=2.5)

    sns.lineplot(ax=axs, 
                 x =  np.arange(365),
                 data = targets[((targets.year==2019) & (targets.playerId==playerid))],
                 y = col,
                 label = '2019',
                 linewidth=2.5)

    sns.lineplot(ax=axs, 
                 x =  np.arange(366),
                 data = targets[((targets.year==2020) & (targets.playerId==playerid))],
                 y = col,
                 label = '2020',
                 linewidth=2.5)

    sns.lineplot(ax=axs, 
                 x =  np.arange(120),
                 data = targets[((targets.year==2021) & (targets.playerId==playerid))],
                 y = col,
                 label = '2021',
                 linewidth=2.5)
    
    bbox = axs.get_position()
    center=0.5*(bbox.x1)
    plt.suptitle(f'player Id {playerid}', x = center)

# There is definitely seasonality for targets, seems like we can remove year 2018 from modelling

In [ ]:
plot_target_for_player()

In [ ]:
plot_target_for_player('target2')

In [ ]:
plot_target_for_player('target3')

In [ ]:
plot_target_for_player('target4')

In [ ]:
plot_target_for_player('target1',477132)

In [ ]:
plot_target_for_player('target2',477132)

In [ ]:
plot_target_for_player('target3',477132)

In [ ]:
plot_target_for_player('target4',477132)

# There seems to be relationship between number of awards and targets, higher awards the player is popular

In [ ]:
from scipy.stats import boxcox
xt, _ = boxcox(targets['target1'].values + 1)
sns.distplot(xt)

In [ ]:
xt, _ = boxcox(targets['target4'].values + 1)
sns.distplot(xt)

In [ ]:
xt, _ = boxcox(targets['target3'].values + 1)
sns.distplot(xt)

**Target4 has highest correlation with Twitter follower count**

we use now ->


['label_playerId', 'label_primaryPositionName', 'label_teamId',
'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
'groundOutsPitching', 'runsPitching', 'doublesPitching',
'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
'inheritedRunnersScored', 'catchersInterferencePitching',
'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
'assists', 'putOuts', 'errors', 'chances','target1_mean',
'target1_median',
'target1_std',
'target1_min',
'target1_max',
'target1_prob','target2_mean','target2_median','target2_std','target2_min',
'target2_max','target2_prob','target3_mean','target3_median',target3_std','target3_min','target3_max','target3_prob','target4_mean','target4_median',
'target4_std','target4_min','target4_max','target4_prob']

correlated(<0.5) ->

groundIntoTriplePlay, caughtStealing, stolenBases, triples, sacBuntsPitching, catchersInterference, pickoffs, gamesPlayedPitching, gamesStartedPitching, completeGamesPitching, shutoutsPitching, winsPitching, lossesPitching, flyOutsPitching, airOutsPitching, groundOutsPitching, runsPitching,
Total doublesPitching, Total triplesPitching, Totla homeRunsPitching, strikeOutsPitching, Total baseOnBallsPitching, 
Total internationalWalksPitching, Total hitsPitching, Total hitByPitchPitching, Total atBatsPitching, TotalcaughtSteallingPitching,
Total stolenBasesPitching, Total inningsPitched, saveOpportunities, earnedRuns, battersFaced, outsPitching, 
Total pitchesThrown, Total balls, Total strikes, Total hitBatsmen, Total balks, Total wildPitches, Total pickoffsPitching, rbiPitching
Total gamesFinishedPitching, Total inheritedRunners,  Total inheritedRunnersScored, Total catchersInferencePitching, sacBuntsPitching, 
Total sacFliesPitching, Total saves, Total holds, Total blownSaves, Total assists, Total putOuts, 


In [ ]:
import seaborn as sns

In [ ]:
player_target_stats = pd.read_csv("../input/player-target-stats/player_target_stats.csv")
targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('median').reset_index()
player_target_stats_for_corr = pd.merge(player_target_stats, targets_agg, on = ['playerId'], how = 'left')
plt.figure(figsize=(20, 20))
plt.xticks(rotation=25)
corr = player_target_stats_for_corr.drop(columns =['playerId']).corr()


sns.heatmap(corr,
            vmin=0,
            vmax=1,
            center= 0,
            cmap="RdYlGn"
       )
plt.title('targets corr')
plt.show()

In [ ]:
followers_agg =followers.groupby('playerId')['numberOfFollowers'].agg('median').reset_index()
targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('median').reset_index()
followers_agg.columns = ['playerId', '#Followers']
followers_agg = pd.merge(followers_agg, targets_agg, on = ['playerId'], how = 'left')
plt.figure(figsize=(10, 2))
plt.xticks(rotation=45)
plt.suptitle("Median Target vs Median Twitter Followers", fontsize =15)

corr = followers_agg.drop(columns =['playerId']).corr()
#mask = np.triu(np.ones_like(corr, dtype=bool))

x_axis_labels = ['#Followers', 'target1','target2', 'target3', 'target4'] 
sns.heatmap(np.array(corr['#Followers']).reshape((1,5)),
            annot = True,
            xticklabels=x_axis_labels,
            vmin=0,
            vmax=1,
            center= 0,
            cmap="RdYlGn"
       )

In [ ]:
tbs = pd.read_csv('../input/mlb-pdef-train-dataset/transactions_train.csv')
tbs.info()

In [ ]:
tbs_cols = tbs.columns.tolist()

tbs_cols = [col for col in tbs_cols if col not in ['playerId', 'playerName', 'fromTeamName',
                                                   'toTeamName', 'effectiveDate', 'resolutionDate',
                                                   'typeCode', 'typeDesc', 'description', 'index']]
for col in tbs_cols:
            tbs_agg = tbs.groupby('playerId')[col].agg('sum').reset_index()
            tbs_agg.columns = ['playerId', "#"+col]
            targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('median').reset_index()
            tbs_agg = pd.merge(tbs_agg, targets_agg, on = ['playerId'], how = 'left')
            
            plt.figure(figsize=(10, 2))
            plt.suptitle(f"Total {col} vs Median targets", fontsize =15)
            corr = tbs_agg.drop(columns =['playerId']).corr()
            mask = np.triu(np.ones_like(corr, dtype=bool))
            sns.set(font_scale=1.4)
            plt.xticks(rotation=45)

            
            x_axis_labels = ["#"+col, 'target1','target2', 'target3', 'target4'] 

            sns.heatmap(np.array(corr["#"+col]).reshape((1,5)),
            annot = True,
            xticklabels=x_axis_labels,
            vmin=0,
            vmax=1,
            center= 0,
            cmap="RdYlGn")           
            
            plt.show()
            plt.close()

In [ ]:
rost_repl = {'A':0, 'D60':1, 'RM':2, 'D10':3, 'D7':4, 'PL':4, 'SU':5, 'FME':6, 'BRV':7, 'RES':8,
       'DEC':9}
rosters.statusCode.replace(rost_repl, inplace=True)

In [ ]:
rosters.info()

In [ ]:
rosters_cols = rosters.columns.tolist()

rosters_cols = [col for col in rosters if col not in ['playerId', 'gameDate', 'status']]
for col in rosters_cols:
            rosters_agg = rosters.groupby('playerId')[col].agg('sum').reset_index()
            rosters_agg.columns = ['playerId', "#"+col]
            targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('median').reset_index()
            rosters_agg = pd.merge(rosters_agg, targets_agg, on = ['playerId'], how = 'left')
            
            plt.figure(figsize=(10, 2))
            plt.suptitle(f"Total {col} vs Median targets", fontsize =15)
            corr = rosters_agg.drop(columns =['playerId']).corr()
            mask = np.triu(np.ones_like(corr, dtype=bool))
            sns.set(font_scale=1.4)
            plt.xticks(rotation=45)

            
            x_axis_labels = ["#"+col, 'target1','target2', 'target3', 'target4'] 

            sns.heatmap(np.array(corr["#"+col]).reshape((1,5)),
            annot = True,
            xticklabels=x_axis_labels,
            vmin=0,
            vmax=1,
            center= 0,
            cmap="RdYlGn")           
            
            plt.show()
            plt.close()

In [ ]:
qcut = pd.qcut(targets['target2'], q=10, duplicates='drop', labels=[1,2,3,4,5,6,7, 8, 9, 10])
top_deciles_index = qcut[qcut == 10].index
target2_years = targets.iloc[top_deciles_index]

In [ ]:
qcut

In [ ]:
target2_years['year'] = pd.to_datetime(targets.loc[top_deciles_index].date, format = '%Y%m%d').dt.year
target2_years['month'] = pd.to_datetime(targets.loc[top_deciles_index].date, format = '%Y%m%d').dt.month
target2_years.groupby('year')['month'].value_counts().sort_index()

In [ ]:
events

In [ ]:
targets

In [ ]:
qcut1 = pd.qcut(targets['target1'], q=10, duplicates='drop', labels=[1,2,3,4,5,6,7])
top_deciles_index1 = qcut1[qcut1 == 7].index
target1_years = targets.iloc[top_deciles_index1]
target1_years['year'] = pd.to_datetime(targets.iloc[top_deciles_index1].date, format = '%Y%m%d').dt.year
target1_years.year.value_counts()

In [ ]:
qcut3 = pd.qcut(targets['target3'], q=10, duplicates='drop', labels=[1,2,3,4,5,6,7])
top_deciles_index3 = qcut3[qcut3 == 7].index
target3_years = targets.iloc[top_deciles_index3]
target3_years['year'] = pd.to_datetime(targets.iloc[top_deciles_index3].date, format = '%Y%m%d').dt.year
target3_years.year.value_counts()

In [ ]:
qcut4 = pd.qcut(targets['target4'], q=10, duplicates='drop', labels=[1,2,3,4,5,6,7,8,9])
top_deciles_index4 = qcut4[qcut4 == 9].index
target4_years = targets.iloc[top_deciles_index4]
target4_years['year'] = pd.to_datetime(targets.iloc[top_deciles_index4].date, format = '%Y%m%d').dt.year
target4_years.year.value_counts()

In [ ]:
awards = pd.read_csv(TRAIN_DIR / 'awards_train.csv')

awards_agg = awards.groupby('playerId')['awardId'].agg('count').reset_index()
awards_agg.columns = ['playerId', '#Awards']
targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('median').reset_index()
followers_agg = pd.merge(awards_agg, targets_agg, on = ['playerId'], how = 'left')
plt.figure(figsize=(10, 2))
plt.xticks(rotation=45)
plt.suptitle("Total number of awards vs Median targets", fontsize =15)
corr = followers_agg.drop(columns =['playerId']).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))

x_axis_labels = ['#Awards', 'target1','target2', 'target3', 'target4'] 
sns.heatmap(np.array(corr['#Awards']).reshape((1,5)),
            annot = True,
            xticklabels=x_axis_labels,
            vmin=0,
            vmax=1,
            center= 0,
            cmap="RdYlGn"
       )

In [ ]:
scores_cols = scores.columns.tolist()

scores_cols = [col for col in scores_cols if col not in ['playerId', 'date', 'home', 'gamePk', 'teamId', 'battingOrder']]
for col in scores_cols:
            scores_agg = scores.groupby('playerId')[col].agg('sum').reset_index()
            scores_agg.columns = ['playerId', "#"+col]
            targets_agg =targets.groupby('playerId')[['target1', 'target2', 'target3', 'target4']].agg('median').reset_index()
            scores_agg = pd.merge(scores_agg, targets_agg, on = ['playerId'], how = 'left')
            plt.figure(figsize=(10, 2))
            plt.suptitle(f"Total {col} vs Median targets", fontsize =15)
            corr = scores_agg.drop(columns =['playerId']).corr()
            mask = np.triu(np.ones_like(corr, dtype=bool))
            sns.set(font_scale=1.4)
            plt.xticks(rotation=45)
            
            x_axis_labels = ["#"+col, 'target1','target2', 'target3', 'target4'] 

            sns.heatmap(np.array(corr["#"+col]).reshape((1,5)),
            annot = True,
            xticklabels=x_axis_labels,
            vmin=0,
            vmax=1,
            center= 0,
            cmap="RdYlGn")           
            
            plt.show()
            plt.close()